In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
!pip install kmodes

In [2]:
persones=pd.read_csv('Accident_people_2021_2022.csv')
persones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17919 entries, 0 to 17918
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    17919 non-null  int64 
 1   File_Num                      17919 non-null  object
 2   District                      17919 non-null  object
 3   Neighbourhood                 17919 non-null  object
 4   Street_name                   17919 non-null  object
 5   Day_name                      17919 non-null  object
 6   year                          17919 non-null  int64 
 7   month                         17919 non-null  int64 
 8   Month_name                    17919 non-null  object
 9   day                           17919 non-null  int64 
 10  Day_shift                     17919 non-null  object
 11  hour                          17919 non-null  int64 
 12  Pedestrian_situation          17919 non-null  object
 13  Vehicle_type    

In [3]:
weather=pd.read_csv('data_meteo_estudifinal_2021_2022.csv')
weather = weather.drop(weather.columns[0], axis=1)
weather.rename(columns={'DATA_LECTURA':'Date'},inplace=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       35040 non-null  object 
 1   03.TM      35040 non-null  float64
 2   06.HRM     35040 non-null  float64
 3   07.PPT24H  35040 non-null  float64
 4   08.HPA     35040 non-null  float64
 5   09.RS24H   35040 non-null  float64
 6   10.VVM10   35040 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.9+ MB


In [4]:
persones= persones[persones['Person_type'] == 'Driver']
persones.head()

,Unnamed: 0,File_Num,District,Neighbourhood,Street_name,Day_name,year,month,Month_name,day,...,Pedestrian_situation,Vehicle_type,Gender,Age,Person_type,Pedestrian_accident_location,Pedestrian_intention,Driver_intention,Accident_severity,Date
6,6,2021S003310,Ciutat Vella,el Barri Gòtic,Catalunya,Dijous,2021,6,Juny,17,...,Not pedestrian fault,Vehicle without motor,Men,31,Driver,A la vorera / Andana ...,Desconegut,Oci i entreteniment,Assistance,2021-06-17 20:00:00
8,8,2021S001728,Ciutat Vella,la Barceloneta,Marina,Diumenge,2021,4,Abril,4,...,Not known,Vehicle without motor,Women,45,Driver,A la vorera / Andana ...,Desconegut,Oci i entreteniment,Hospitalized,2021-04-04 10:00:00
17,17,2021S005721,Eixample,el Fort Pienc,Marina,Dimecres,2021,10,Octubre,13,...,Not pedestrian fault,Other,Women,51,Driver,A la vorera / Andana ...,Desconegut,In itínere,Hospitalized,2021-10-13 08:00:00
31,31,2021S005403,Eixample,l'Antiga Esquerra de l'Eixample,Aragó,Dimecres,2021,9,Setembre,29,...,Not pedestrian fault,Car,Men,71,Driver,A la vorera / Andana ...,Desconegut,Oci i entreteniment,Assistance,2021-09-29 09:00:00
39,39,2021S004164,Eixample,Sant Antoni,Comte Borrell,Divendres,2021,7,Juliol,23,...,Not pedestrian fault,Motorcycle,Men,24,Driver,A la vorera / Andana ...,Desconegut,Altres activitats,Hospitalized,2021-07-23 19:00:00


In [5]:
#FROM ALL THE DATA WE ARE JUST GOING TO FOCUS N¡ON THE DRIVERS BECAUSE
#THE PASSANGER OR THE PEDESTRIAN DOESN0T HAVE ANY KIND OF POWER TO NOT BE IN THE SITUATION OR HAVE ANY CONTROL

persones=persones[['Day_shift','Vehicle_type','Gender','Age','Date']]
weather=weather[['Date','03.TM','06.HRM','10.VVM10']]

#JOIN THE DATASETS WITH THE VARIABLE DATE
variables = pd.merge(persones, weather, on='Date', how='left')
variables.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12681 entries, 0 to 12680
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Day_shift     12681 non-null  object 
 1   Vehicle_type  12681 non-null  object 
 2   Gender        12681 non-null  object 
 3   Age           12681 non-null  object 
 4   Date          12681 non-null  object 
 5   03.TM         12681 non-null  float64
 6   06.HRM        12681 non-null  float64
 7   10.VVM10      12681 non-null  float64
dtypes: float64(3), object(5)
memory usage: 891.6+ KB


In [6]:
#add variable quarter and start studying the different variables
variables['Quarter'] = variables['Date'].apply(lambda x: pd.Timestamp(x).quarter)
variables.head()

,Day_shift,Vehicle_type,Gender,Age,Date,03.TM,06.HRM,10.VVM10,Quarter
0,Afternoon,Vehicle without motor,Men,31,2021-06-17 20:00:00,23.4,97.0,1.1,2
1,Morning,Vehicle without motor,Women,45,2021-04-04 10:00:00,15.9,55.0,2.3,2
2,Morning,Other,Women,51,2021-10-13 08:00:00,19.3,65.0,1.0,4
3,Morning,Car,Men,71,2021-09-29 09:00:00,23.5,64.0,1.4,3
4,Afternoon,Motorcycle,Men,24,2021-07-23 19:00:00,27.7,53.0,0.8,3


In [7]:
# LETS TRANSFORM TO CATHEGORIC THE COLUMN EDAT
#/// desconegut(404) -->NOT KNOWN
#transform value desconegut a 404
variables['Age'] = variables['Age'].replace('Desconegut', '404')
def categorize_age(age):
    if int(age) <= 25:
        return '16-25'
    elif int(age) <= 35 and int(age) > 25:
        return '26-35'
    elif int(age) <= 45 and int(age) > 35:
        return '36-45'
    elif int(age) <= 55 and int(age) > 45:
        return '46-55'
    elif age == '404':
        return 'Age not known'
    else:
        return '+56'

# Apply function to transform ages to categories
variables['Age'] = variables['Age'].apply(categorize_age)

x=variables['Age'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['Age'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['26-35' '36-45' '46-55' '+56' '16-25' 'Age not known']
26-35            29.642773
36-45            23.042347
16-25            17.916568
46-55            17.569592
+56              11.781405
Age not known     0.047315
Name: Age, dtype: float64


In [8]:
#x=variables['03.TM'].unique()
# print(np.sort(x)) the data goes from 3.8 to 34.2
def cat_temp(temp):
    if int(temp) <= 13:
        return 'Very low'
    elif int(temp) <= 17 and int(temp) > 13:
        return 'Low'
    elif int(temp) <= 24 and int(temp) > 17:
        return 'Mild'
    else:
        return 'Hot'
variables['03.TM'] = variables['03.TM'].apply(cat_temp)

x=variables['03.TM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['03.TM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Mild' 'Low' 'Hot' 'Very low']
Mild        30.273638
Very low    25.045343
Hot         23.200063
Low         21.480956
Name: 03.TM, dtype: float64


In [9]:
#x=variables['06.HRM'].unique()
#print(np.sort(x)) from 9 percent to 100 percent
def cat_humidity(hum):
    if int(hum) <= 48:
        return 'Low humidity'
    elif int(hum) <= 60 and int(hum) > 48:
        return 'Moderate humidity'
    elif int(hum) <= 75 and int(hum) > 60:
        return 'High humidity'
    else:
        return 'Very high'
variables['06.HRM'] = variables['06.HRM'].apply(cat_humidity)

x=variables['06.HRM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['06.HRM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Very high' 'Moderate humidity' 'High humidity' 'Low humidity']
High humidity        39.216150
Moderate humidity    25.234603
Low humidity         19.075783
Very high            16.473464
Name: 06.HRM, dtype: float64


In [10]:
#x=variables['10.VVM10'].unique()
#print(np.sort(x)) from 0 to 8.7
def cat_wind(wind):
    if int(wind) <= 1:
        return 'Low speed'
    elif int(wind) <= 2 and int(wind) > 1:
        return 'Moderate speed'
    else:
        return 'High speed'
    
variables['10.VVM10'] = variables['10.VVM10'].apply(cat_wind)

x=variables['10.VVM10'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['10.VVM10'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Low speed' 'Moderate speed' 'High speed']
Low speed         43.080199
Moderate speed    30.344610
High speed        26.575191
Name: 10.VVM10, dtype: float64


In [11]:
from itertools import combinations
from sklearn import preprocessing
from kmodes.kmodes import KModes
# List of variable names
variables_name = ['Day_shift', 'Vehicle_type', 'Gender', 'Age']

# Generate combinations of 4 to 8 variables
combinations_list = []
for r in range(4, 5):
    combinations_list.extend(combinations(variables_name, r))

# Empty DataFrame to store results
table = pd.DataFrame()

def tuple_to_list(t):
    new_list = []
    for element in t:
        new_list.append(element)
    return new_list

# Iterate over combinations
for selected_columns in combinations_list:
    # Select columns from 'variables' DataFrame based on the combination
    listavar = tuple_to_list(selected_columns)
    subgrup_1 = variables[listavar]
    print(subgrup_1.shape[1])
    # Clustering code
    subgroup = subgrup_1.copy()
    from sklearn import preprocessing  
    le = preprocessing.LabelEncoder()
    subgroup_encoded = subgroup.apply(le.fit_transform)
    #to create the list to get the percenge of significant clusters
    l=[]
    # Iterate through cluster numbers
    for n_clusters in range(5,6):
        # Clustering
        km = KModes(n_clusters=n_clusters, init='Cao', n_init=1, verbose=1)
        clusters = km.fit_predict(subgroup_encoded)

        # Combine with original data
        combined = pd.concat([subgroup.reset_index(), pd.DataFrame({'Cluster': clusters})], axis=1).drop('index', axis=1)

        # Create table
        table = pd.DataFrame(columns=range(n_clusters))

        # Fill in table with percentage of instances of each value in each cluster for each variable
        for var in subgroup.columns:
            by_cluster_value = combined.groupby(['Cluster', var]).size()
            percentages = by_cluster_value.groupby(level=0).apply(lambda x: x / x.sum() * 100)
            pivot_table = pd.pivot_table(percentages.reset_index(), index=var, columns='Cluster', values=0, fill_value=0)

            # Fill in table with percentage of instances for each value in each cluster
            for i, col in enumerate(pivot_table.columns):
                values = pivot_table[col].index.tolist()
                pct_values = pivot_table[col].tolist()
                for j, value in enumerate(values):
                    table.loc[value, i] = pct_values[j]
            # Add new column with variable names based on index
        table['VARIABLE'] = ''

        variable_mapping = {}

        for column in subgroup.columns:
            unique_values = subgroup[column].unique()
            variable_mapping[column] = unique_values

        for i, index_value in enumerate(table.index.values):
            for variable, values in variable_mapping.items():
                if index_value in values:
                    table.iloc[i, -1] = variable
                    break
        print(table)
        table = table.reset_index().rename(columns={'index': 'value'})
        table = table[['VARIABLE'] + list(table.columns[:-1])]
        #print(table)
        table = table.set_index('VARIABLE')       

        # Add a new row for the cluster drivers percentage
        table.loc['Cluster percentage of drivers'] = 0

        # Calculate the number of instances in each cluster
        cluster_counts = combined['Cluster'].value_counts()

        # Fill in the table with the percentage of instances in each cluster
        for i, count in cluster_counts.items():
            table.loc['Cluster percentage of drivers', i] = count / len(combined) * 100
    
        #table.to_excel(f'table_{n_clusters}_clusters_pas1.xlsx', index=True)
        table1=table
        # Save table to CSV file
        table = table.drop('value', axis=1)
        new_table_2 = pd.DataFrame(index=table.index.unique())
    # Iterate over each column (0, 1, 2, 3) and find the highest percentage for each variable
        for col in table.columns:
            col_max = table.groupby(level=0)[col].max()
            new_table_2[col] = col_max
    # Add the 'Cluster percentage of drivers' row back to the new DataFrame
        new_table_2.loc['Cluster percentage of drivers'] = table.loc['Cluster percentage of drivers']
    # Display the new DataFrame
        #print(new_table_2)
        #new_table_2.to_excel(f'table_{n_clusters}_clusters_pas2.xlsx', index=True)
        new_table_3= new_table_2.loc[new_table_2.index != 'Cluster percentage of drivers'] = (new_table_2.loc[new_table_2.index != 'Cluster percentage of drivers'] >= 75).astype(int)
        new_table_3.loc['VARIABLES => 75%'] = new_table_3.sum()
        #new_table_2.to_excel(f'table_{n_clusters}_clusters_pas3.xlsx', index=True)
        #print(new_table_3)
        print(subgrup_1.shape[1])  
        new_table_3.loc['Cluster Value'] = new_table_3 .loc['VARIABLES => 75%'] / subgrup_1.shape[1]
        new_table_3.loc['Cluster adequate?'] = new_table_3 .loc['Cluster Value']>0.5
        new_table_4=new_table_3
        new_table_4.loc['Cluster percentage of drivers'] = table.loc['Cluster percentage of drivers']
        new_table_4.loc['SCORE'] = np.nan
        new_table_4.loc['SCORE' ,'TOTAL CLUSTERIZATION VALUE'] = new_table_4.loc['Cluster adequate?'].sum() / n_clusters
        if new_table_4.loc['Cluster adequate?'].sum() / n_clusters > 0.9:
            table1.to_excel(f'table_{n_clusters}_clusters_{"".join(selected_columns)}.xlsx', index=True)
       
    
    

4
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 14958.0
                               0          1          2          3          4  \
Afternoon              81.225735  17.099651  10.200669        0.0  19.290123   
Morning                10.385469  77.782086  79.765886   92.06501  18.518519   
Night                   8.388796   5.118263  10.033445    7.93499  62.191358   
Car                    10.108153  12.058938   53.26087        0.0   5.246914   
Motorcycle             69.855796  66.382319  24.749164  86.042065  23.765432   
Other                  10.648918  10.081427   11.87291     8.3174  68.055556   
Vehicle without motor   9.387133  11.477317  10.117057   5.640535   2.932099   
Men                    77.066001  20.395502  90.719064        100  92.901235   
Not known               0.027732        0.0        0.0          0   0.154321   
Women                  22.906267  79.604498   9.280936          0   6

C:\Users\paufg\AppData\Local\Temp\ipykernel_17836\771656300.py:49: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  percentages = by_cluster_value.groupby(level=0).apply(lambda x: x / x.sum() * 100)
C:\Users\paufg\AppData\Local\Temp\ipykernel_17836\771656300.py:49: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_